# 01 - scRNA-seq iHSC (Ng et al. 2025): QC y clustering

Este notebook realiza el control de calidad y el clustering inicial de las células individuales.


In [ ]:
import os
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt

FIG_DIR = '../results/figures/'
TABLE_DIR = '../results/tables/'
os.makedirs(FIG_DIR, exist_ok=True)
os.makedirs(TABLE_DIR, exist_ok=True)

# Cargar objeto procesado (modificar ruta según tu archivo)
adata = sc.read_h5ad('../data_processed/iHSC_scRNAseq_Ng2025.h5ad')

# QC
sc.pp.calculate_qc_metrics(adata, inplace=True)
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'], jitter=0.4, show=False)
plt.savefig(FIG_DIR + '01_QC_violin.png', dpi=300, bbox_inches='tight')
plt.close()

# Normalización / clustering (adaptar si ya está hecho)
# sc.pp.normalize_total(adata, target_sum=1e4)
# sc.pp.log1p(adata)
# sc.pp.highly_variable_genes(adata, n_top_genes=3000, subset=True)
# sc.pp.scale(adata, max_value=10)
# sc.tl.pca(adata)
# sc.pp.neighbors(adata)
# sc.tl.umap(adata)
# sc.tl.leiden(adata, key_added='cluster')

# UMAP por cluster
sc.pl.umap(adata, color=['cluster'], show=False)
plt.savefig(FIG_DIR + '01_umap_clusters.png', dpi=300, bbox_inches='tight')
plt.close()

# Ejemplo UMAP por condición (si existe la columna)
if 'retaa_regimen' in adata.obs.columns:
    sc.pl.umap(adata, color=['retaa_regimen'], show=False)
    plt.savefig(FIG_DIR + '01_umap_RETA_regimen.png', dpi=300, bbox_inches='tight')
    plt.close()

# Marcadores por cluster (opcional)
try:
    sc.tl.rank_genes_groups(adata, 'cluster', method='wilcoxon')
    sc.pl.rank_genes_groups(adata, n_genes=20, sharey=False, show=False)
    plt.savefig(FIG_DIR + '01_marker_genes_clusters.png', dpi=300)
    plt.close()
    markers_df = pd.DataFrame(adata.uns['rank_genes_groups']['names'])
    markers_df.to_csv(TABLE_DIR + '01_markers_per_cluster.csv')
except Exception as e:
    print('No se pudieron calcular marcadores por cluster:', e)

# Guardar versión QC/normalizada
adata.write('../data_processed/iHSC_scRNAseq_Ng2025_qc_norm.h5ad')
